# 1. Instalar NLP API's

In [1]:
#!pip install -U spacy

In [2]:
#!python -m spacy download es

In [3]:
#!pip install nltk
#import nltk
#nltk.download('stopwords')

In [4]:
#!pip install circlify


In [5]:
#!pip install seaborn

In [6]:
#!pip install spacy_syllables

# 2. Instalar de modelo en Español

In [7]:
import os
import sys
PATH = os.getcwd()
PATH = PATH + '{0}'.format(os.sep)
LOGIC = PATH + '{0}logic{0}'.format(os.sep)
DIR_DATA = PATH + '{0}data{0}'.format(os.sep)
sys.path.append(PATH) if PATH not in list(sys.path) else None
sys.path.append(DIR_DATA) if DIR_DATA not in list(sys.path) else None
print(PATH, "\n", LOGIC, "\n", DIR_DATA)

C:\Users\epuerta\OneDrive - Universidad Tecnológica de Bolívar\Apps\AppsISCO\CienciaDatos-DING-3064\ 
 C:\Users\epuerta\OneDrive - Universidad Tecnológica de Bolívar\Apps\AppsISCO\CienciaDatos-DING-3064\\logic\ 
 C:\Users\epuerta\OneDrive - Universidad Tecnológica de Bolívar\Apps\AppsISCO\CienciaDatos-DING-3064\\data\


# 3. Importar librerias

In [8]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter

# Plot libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Text analysis API's
import re
import spacy
import nltk
import unicodedata
from nltk.corpus import stopwords
from nltk import TweetTokenizer
from spacy.lang.es import Spanish

# Preprocessin API´s
from sklearn.preprocessing import LabelEncoder 
from sklearn import preprocessing

# Regularization API´s 
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit

# Feature extration API´s
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from logic.text_processing import TextProcessing
from logic.lexical_vectorizer import LexicalVectorizer

# Machine Learning Methods 

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import tree

# Metrics Libraries
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score


# 4. Inicializar spaCy y cargar modelo

In [9]:
nlp = spacy.load('es_core_news_sm')

# 5. Importar Dataset

TASS-2018: Workshop on Semantic Analysis at SEPLN

In [10]:
ftrain = '{0}{1}{2}'.format(DIR_DATA, 'TASS2018', '.csv')

In [11]:
data_raw = pd.read_csv(ftrain, sep=';')
data_raw.head(10)

,tweetid,user,content,date,lang,polarity
0,770976639173951488,noseashetero,@noseashetero 1000/10 de verdad a ti que voy a...,31/08/2016 13:28,es,P
1,771092421866389508,Templelx,@piscolabisaereo @HistoriaNG @SPosteguillo las...,31/08/2016 21:08,es,P
2,771092111429083136,esskuu94,"Al final han sido 3h Bueno, mañana tengo fies...",31/08/2016 21:07,es,P
3,771092070572449796,__ariadna9,@Jorge_Ruiz14 yo no tengo tiempo para esas cos...,31/08/2016 21:07,es,N
4,771094192508600320,_cristtina15_,@_MissChaotic_ ves ese brillo? es un coso que ...,31/08/2016 21:15,es,N
5,771116956518875137,JSorgex,Tengo una perrina adorable... Sabéis que me ac...,31/08/2016 22:46,es,P
6,771115324884262912,Escarolilla,@juankipua Es que en el Ojeando el año pasado ...,31/08/2016 22:39,es,NEU
7,771118683414560768,KaichiZick95,"Bueno, estoy en la batalla final del Conquista...",31/08/2016 22:53,es,NONE
8,770550748107202560,carlosnmr,@CondeDuqueMAD ¿mañana sábado 31? En que día v...,30/08/2016 9:16,es,NONE
9,770549862500802560,hermesni,"@agcasti y el caminante ante el mar de niebla,...",30/08/2016 9:12,es,P


# 6. Describir Dataset

In [ ]:
data_raw.info()

# 7. Descripción del contenido 

## 7.1 Serie de tiempo de tweets

In [ ]:
data_raw['hour'] = pd.DatetimeIndex(data_raw['date']).hour
data_raw['minute'] = pd.DatetimeIndex(data_raw['date']).minute
fig = px.line(data_raw, x='polarity', y='hour') 
fig.show()

## 7.2 Descripción de la polaridad 

In [ ]:
tweet_by_polarity = data_raw.groupby("polarity", as_index=False)['content'].count()
tweet_by_polarity.head(10)

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(data =tweet_by_polarity, x = "polarity", y = "content", alpha=0.8)
plt.title('Polary Frequency by content')
plt.show()

## 7. 3 Uso de palabras en tweets

In [ ]:
words = {}
for row in tqdm(data_raw['content']):
    doc = nlp(row.lower())
    for token in doc:
        if token.is_alpha and not token.is_stop:
            if token.text in words:
                num_temp = int(words[token.text])
                words[token.text] = num_temp + 1
            else:
                words[token.text] = 1


In [ ]:
sorted_words = sorted(words.items(), key=lambda x:x[1], reverse=True)
sorted_words

In [ ]:
df_words = pd.DataFrame(data = sorted_words, columns=['Word', 'Freq'])
df_words = df_words[:20]
df_words.head(10)

In [ ]:
def get_colordict(palette,number,start):
    pal = list(sns.color_palette(palette=palette, n_colors=number).as_hex())
    color_d = dict(enumerate(pal, start=start))
    return color_d

In [ ]:
import circlify
# compute circle positions:
circles = circlify.circlify(df_words['Freq'][0:30].tolist(), 
                            show_enclosure=False, 
                            target_enclosure=circlify.Circle(x=0, y=0)
                           )
n = df_words['Freq'][0:30].max()
color_dict = get_colordict('RdYlBu_r',n ,1)

In [ ]:
fig, ax = plt.subplots(figsize=(9,9), facecolor='white')
ax.axis('off')
lim = max(max(abs(circle.x)+circle.r, abs(circle.y)+circle.r,) for circle in circles)
plt.xlim(-lim, lim)
plt.ylim(-lim, lim)

# list of labels
labels = list(df_words['Word'][0:40])
counts = list(df_words['Freq'][0:40])
labels.reverse()
counts.reverse()

# print circles
for circle, label, count in zip(circles, labels, counts):
    x, y, r = circle
    ax.add_patch(plt.Circle((x, y), r, alpha=0.9, color = color_dict.get(count)))
    plt.annotate(label +'\n'+ str(count), (x,y), size=12, va='center', ha='center')
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
pos_freq = {}
for row in tqdm(data_raw['content'].to_list()):
    doc = nlp(row.lower())
    for token in doc:
        if token.pos_ in pos_freq:
            value = pos_freq[token.pos_]
            pos_freq[token.pos_] = value + 1
        else:
            pos_freq[token.pos_] =  1

In [ ]:
df_pos = pd.DataFrame([[key, pos_freq[key]] for key in pos_freq.keys()], columns=['POS', 'Freq'])
df_pos.sort_values('Freq').tail(15)
df_pos = df_pos[:20]
df_pos.head(10)

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(data = df_pos, x = 'POS', y = 'Freq', alpha=0.8)
plt.title('POS Frequency')
plt.show()

# 8. Preprocesamiento

In [ ]:
tp = TextProcessing(lang='es')

In [ ]:
messages = [tp.clean(row, stopwords=True) for row in data_raw['content'].tolist()]
messages

In [ ]:
lexical_vector = LexicalVectorizer()

In [ ]:
x = lexical_vector.transform(messages)
x

In [ ]:
df = pd.DataFrame(x, index=['tweet '+str(i) for i in range(1, 1+len(messages))])
df.round(3)

In [ ]:
y = data_raw['polarity']
y

In [ ]:
Counter(y)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x, y, test_size=0.3, random_state=42)

In [ ]:
oversample = SMOTE()
xx_train, yy_train = oversample.fit_resample(x_train, y_train)
xx_test, yy_test = oversample.fit_resample(x_test, y_test)

In [ ]:
# Train
Counter(y_train)

In [ ]:
# Test
Counter(yy_test)

In [ ]:
k_fold = ShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

In [ ]:
models = [("DT", DecisionTreeClassifier(max_depth=70, min_samples_leaf=1, min_samples_split=3)),
          ("SVM", SVC(kernel='linear', C=0.5, probability=True)),
          ("RF", RandomForestClassifier(n_estimators=200, max_depth=6, random_state=0)),
          ("NB", GaussianNB())]

In [ ]:
finalResults = []
cmList = []
for name, model in models:
    accuracies = []
    precisions = []
    recalls = []
    f1s = []
    for train_index, test_index in k_fold.split(xx_train, yy_train):
        data_train = xx_train[train_index]
        target_train = yy_train[train_index]

        data_test = xx_train[test_index]
        target_test = yy_train[test_index]

        model.fit(data_train, target_train)
        predict = model.predict(data_test)
        # Accuracy
        accuracy = accuracy_score(target_test, predict, normalize=True)
        accuracies.append(accuracy)
        # Precision
        precision = precision_score(target_test, predict, average="macro")
        precisions.append(precision)
        # recall
        recall = recall_score(target_test, predict, average="macro")
        recalls.append(recall)
        # f1
        f1 = f1_score(target_test, predict, average="macro")
        f1s.append(f1)
    
    y_test_predict = model.predict(xx_test)
    cm= confusion_matrix(yy_test, y_test_predict)
    
    finalResults.append({'name':name, 
                         'model': model,
                         'accuracy': round(np.mean(accuracies), 2), 
                         'precision': round(np.mean(precisions), 2),
                         'recall': round(np.mean(recalls), 2),
                         'f1': round(np.mean(f1s), 2),
                         'confusion_matrix': cm
                        })
    cmList.append((name,cm))

In [ ]:
df_result = pd.DataFrame.from_dict(finalResults)
df_result

In [ ]:
for name , i in cmList:
    plt.figure()
    sns.heatmap(i , annot =True, linewidth=0.8,fmt=".1f")
    plt.title(name)
    plt.show()